<a href="https://colab.research.google.com/github/stbalaji/langchain/blob/main/RAG_LAB_Volume_Sheet_Aditi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### <font color=FF595E>Installing packages</font>



In [ ]:
! pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain pypdf rapidocr-onnxruntime streamlit unstructured pdf2image pdfminer.six pikepdf pillow_heif langchain_experimental


#### <font color=FF595E>OpenAI API


In [ ]:
#OpenAI API key
from google.colab import userdata
import os
os.environ["OPENAI_API_KEY"] = userdata.get('OPEN_AI_KEY')

In [ ]:
#Setup LangSmith to trace development
from langsmith import Client
os.environ["LANGCHAIN_PROJECT"] = 'RAG_LAB'
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')
os.environ["LANGCHAIN_TRACING_V2"] = "true"

## <font color=FF595E>Creating ChatBot</font>


#### Define models

In [ ]:
GPT4 = 'gpt-4-0125-preview'
GPT3 = 'gpt-3.5-turbo-0125'

#### Simple ChatBot, no memory

In [ ]:
#Import ChatOpenAI class
from langchain_openai import ChatOpenAI


In [ ]:
#Define the LLM. Specify model
Chat = ChatOpenAI(model = GPT4)

In [ ]:
# Invoke the chat with simple question to test it out
Chat.invoke('What is your knowledge cut off day?')

AIMessage(content="My knowledge is up to date until September 2021. I don't have information on events or developments that occurred after that time.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 15, 'total_tokens': 42, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0125-preview', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a8c6f675-c6eb-442b-ad2b-4093c71662f3-0', usage_metadata={'input_tokens': 15, 'output_tokens': 27, 'total_tokens': 42, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

#### Adding memory and memory management

In [ ]:
# Import ChatMessageHistory class that will store our chat histor.
# Import chat prompt templates classes and Message placeholders classes
from langchain.memory import ChatMessageHistory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

In [ ]:
# Initialize a new ChatMessageHistory object
chat_history = ChatMessageHistory()

In [ ]:
# Add a user message to the chat history
chat_history.add_user_message("What day ChatGPT was launched")

In [ ]:
# Add an AI response message to the chat history
chat_history.add_ai_message("ChatGPT was launched at November 30, 2022")


In [ ]:
# Access the messages property of the chat_history object
chat_history.messages

[HumanMessage(content='What day ChatGPT was launched', additional_kwargs={}, response_metadata={}),
 AIMessage(content='ChatGPT was launched at November 30, 2022', additional_kwargs={}, response_metadata={})]

In [ ]:
# Add another user message to the chat history
chat_history.add_user_message("Was it a successful launch?")

In [ ]:
# Create a ChatPromptTemplate using messages
prompt = ChatPromptTemplate.from_messages(
    [
        # Define a system message as a tuple
        (
            "system",
            "You are a data analyst. You need to check the csv file related to volume and data. Total column refers to total number of records also known as total voloume of records or just total volume. The document contains differents status- Total (Total Records Processed), Completed (Success), Failed, Terminated records. This is provided for the entitre year 2023 for each month separately. Tenant refers to the client and workflow refers to a specific workflow. One tenant can have multiple workflows as stated. You need to prepare bar graphs for line charts for the questions asked by the user",
        ),
        # Add a placeholder for the chat messages
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [ ]:
# Create a simple Chain by passing prompt to LLM
Chain = prompt | Chat

In [ ]:
#Invoking simple chain from messages
Chain.invoke({"messages": chat_history.messages})

AIMessage(content='Yes, the launch of ChatGPT was highly successful. It quickly garnered widespread attention for its ability to generate human-like text based on the prompts it was given. Users found its responses to be surprisingly coherent and insightful across a wide range of topics, from casual conversation to complex technical explanations. The launch showcased the capabilities of large language models and their potential applications, leading to a significant increase in interest and discussion around AI and natural language processing technologies. The success was measured not just by the immediate user engagement and feedback, but also by the broader impact it had on promoting discussions around AI ethics, potential use cases, and future developments in the field.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 129, 'prompt_tokens': 164, 'total_tokens': 293, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tok

#### Creating a loop to run Chat with history, printable user imputs and chat outputs

In [ ]:
#Import
from langchain_core.runnables.history import RunnableWithMessageHistory
#Define stop words for our chatbot
stop_words = ["exit", "quit", "stop"]

In [ ]:
#Define chat history
chat_history = ChatMessageHistory()

#Define LLM

Chat = ChatOpenAI(model = GPT4)

# Create a ChatPromptTemplate using messages
prompt = ChatPromptTemplate.from_messages(
    [
        # Define a system message as a tuple
        (
            "system",
            "You are a data analyst. You need to check the csv file related to volume and data. Total column refers to total number of records also known as total voloume of records or just total volume. The document contains differents status- Total (Total Records Processed), Completed (Success), Failed, Terminated records. This is provided for the entitre year 2023 for each month separately. Tenant refers to the client and workflow refers to a specific workflow. One tenant can have multiple workflows as stated. You need to prepare bar graphs for line charts for the questions asked by the user",
        ),
        # Add a placeholder for the chat messages
        MessagesPlaceholder(variable_name="messages"),
    ]
)

#Define the chain
Chat_chain = prompt | Chat

#Use RunnableWithMessageHistory as a wrapper to manage message history
Chain_with_message_history = RunnableWithMessageHistory(
    Chat_chain,
    #define access to chat history
    lambda session_id : chat_history,
    input_messages_key="messages",
    history_messages_key="chat_history"
)

# Perform chat turns
print("Starting the chat...")
while True:
    question = input("User: ")

    # Check if the user input matches a stop word
    if question.lower() in stop_words:
        print("Exiting the chat...")
        break

    # Add a user message to the chat history
    chat_history.add_user_message(question)

    #Generate AI response
    ai_response = Chain_with_message_history.invoke({"messages": chat_history.messages}, {"configurable": {"session_id": chat_history}})

    # Add an AI response message to the chat history
    chat_history.add_ai_message(ai_response.content)

    #Display AI answer
    print(f"AI: {ai_response.content}")

Starting the chat...
User: exit
Exiting the chat...


## <font color=FF595E>Building RAG Chatbot</font>

#### <font color=FF595E>Load documents</font>

In [ ]:
from langchain_community.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path="/content/VolumeSheet.csv")

data = loader.load()

print(data)



[Document(metadata={'source': '/content/VolumeSheet.csv', 'row': 0}, page_content='Tenant: Apollo\nWorkflow: Claims\nStatus: Total\nJan 2023: 439\nFeb 2023: 439\nMar 2023: 1099\nApr 2023: 194\nMay 2023: 200\nJun 2023: 161\nJul 2023: 198\nAug 2023: 153\nSep 2023: 194\nOct 2023: 200\nNov 2023: 161\nDec 2023: 198'), Document(metadata={'source': '/content/VolumeSheet.csv', 'row': 1}, page_content='Tenant: Apollo\nWorkflow: Claims\nStatus: Success\nJan 2023: 239\nFeb 2023: 239\nMar 2023: 899\nApr 2023: 192\nMay 2023: 164\nJun 2023: 103\nJul 2023: 152\nAug 2023: 149\nSep 2023: 192\nOct 2023: 164\nNov 2023: 103\nDec 2023: 152'), Document(metadata={'source': '/content/VolumeSheet.csv', 'row': 2}, page_content='Tenant: Apollo\nWorkflow: Claims\nStatus: Failed\nJan 2023: 100\nFeb 2023: 100\nMar 2023: 100\nApr 2023: 0\nMay 2023: 0\nJun 2023: 0\nJul 2023: 0\nAug 2023: 0\nSep 2023: 0\nOct 2023: 0\nNov 2023: 0\nDec 2023: 0'), Document(metadata={'source': '/content/VolumeSheet.csv', 'row': 3}, page_c

In [ ]:
#Print data to check it out
print(data)

[Document(metadata={'source': '/content/VolumeSheet.csv', 'row': 0}, page_content='Tenant: Apollo\nWorkflow: Claims\nStatus: Total\nJan 2023: 439\nFeb 2023: 439\nMar 2023: 1099\nApr 2023: 194\nMay 2023: 200\nJun 2023: 161\nJul 2023: 198\nAug 2023: 153\nSep 2023: 194\nOct 2023: 200\nNov 2023: 161\nDec 2023: 198'), Document(metadata={'source': '/content/VolumeSheet.csv', 'row': 1}, page_content='Tenant: Apollo\nWorkflow: Claims\nStatus: Success\nJan 2023: 239\nFeb 2023: 239\nMar 2023: 899\nApr 2023: 192\nMay 2023: 164\nJun 2023: 103\nJul 2023: 152\nAug 2023: 149\nSep 2023: 192\nOct 2023: 164\nNov 2023: 103\nDec 2023: 152'), Document(metadata={'source': '/content/VolumeSheet.csv', 'row': 2}, page_content='Tenant: Apollo\nWorkflow: Claims\nStatus: Failed\nJan 2023: 100\nFeb 2023: 100\nMar 2023: 100\nApr 2023: 0\nMay 2023: 0\nJun 2023: 0\nJul 2023: 0\nAug 2023: 0\nSep 2023: 0\nOct 2023: 0\nNov 2023: 0\nDec 2023: 0'), Document(metadata={'source': '/content/VolumeSheet.csv', 'row': 3}, page_c

#### <font color=FF595E>Split the document</font>

In [ ]:
# Import text splitter
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Create an instance of RecursiveCharacterTextSplitter with custom chunk size and overlap
chunk_size = 750  # Adjust the chunk size as needed
chunk_overlap = 0  # Set the overlap between chunks

#Initiate splitter with desired parameters
splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

In [ ]:
# Split the document into chunks using the RecursiveCharacterTextSplitter
splits = splitter.split_documents(data)


In [ ]:
splits = splitter.split_documents(data)


In [ ]:
# Print the number of splits in the doc
print(len(splits))

39


In [ ]:
# Print each split and a separator for readability
for split in splits:
    print(split)
    print("---")

page_content='Tenant: Apollo
Workflow: Claims
Status: Total
Jan 2023: 439
Feb 2023: 439
Mar 2023: 1099
Apr 2023: 194
May 2023: 200
Jun 2023: 161
Jul 2023: 198
Aug 2023: 153
Sep 2023: 194
Oct 2023: 200
Nov 2023: 161
Dec 2023: 198' metadata={'source': '/content/VolumeSheet.csv', 'row': 0}
---
page_content='Tenant: Apollo
Workflow: Claims
Status: Success
Jan 2023: 239
Feb 2023: 239
Mar 2023: 899
Apr 2023: 192
May 2023: 164
Jun 2023: 103
Jul 2023: 152
Aug 2023: 149
Sep 2023: 192
Oct 2023: 164
Nov 2023: 103
Dec 2023: 152' metadata={'source': '/content/VolumeSheet.csv', 'row': 1}
---
page_content='Tenant: Apollo
Workflow: Claims
Status: Failed
Jan 2023: 100
Feb 2023: 100
Mar 2023: 100
Apr 2023: 0
May 2023: 0
Jun 2023: 0
Jul 2023: 0
Aug 2023: 0
Sep 2023: 0
Oct 2023: 0
Nov 2023: 0
Dec 2023: 0' metadata={'source': '/content/VolumeSheet.csv', 'row': 2}
---
page_content='Tenant: Apollo
Workflow: Claims
Status: Needs Attention
Jan 2023: 100
Feb 2023: 100
Mar 2023: 100
Apr 2023: 2
May 2023: 36
Jun 

#### <font color=FF595E>Create embeddings</font>

In [ ]:
#Import vectorstore database and embeddings model
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

# Embeddings model
embeddings_model = OpenAIEmbeddings

In [ ]:
#Define vector DB. Run this line of code only once.
#If accidently did more delete DB

vector_db = Chroma.from_documents(documents=splits, embedding=embeddings_model())

In [ ]:
#Code to delete db. (if needed)

# Delete the collection
vector_db.delete_collection()
print("Collection deleted successfully.")

Collection deleted successfully.


#### <font color=FF595E>Define Retriever</font>

In [ ]:
retriever = vector_db.as_retriever()

#### <font color=FF595E>Test retriever</font>

In [ ]:
#Define question
question = 'What is the volume of total transactions for Apollo for the year 2023?'

#Fetch 3 documents from vector store related to question
vector_db.similarity_search_with_score(question, k=3)

[(Document(metadata={'row': 0, 'source': '/content/VolumeSheet.csv'}, page_content='Tenant: Apollo\nWorkflow: Claims\nStatus: Total\nJan 2023: 439\nFeb 2023: 439\nMar 2023: 1099\nApr 2023: 194\nMay 2023: 200\nJun 2023: 161\nJul 2023: 198\nAug 2023: 153\nSep 2023: 194\nOct 2023: 200\nNov 2023: 161\nDec 2023: 198'),
  0.2737162411212921),
 (Document(metadata={'row': 0, 'source': '/content/VolumeSheet.csv'}, page_content='Tenant: Apollo\nWorkflow: Claims\nStatus: Total\nJan 2023: 439\nFeb 2023: 439\nMar 2023: 1099\nApr 2023: 194\nMay 2023: 200\nJun 2023: 161\nJul 2023: 198\nAug 2023: 153\nSep 2023: 194\nOct 2023: 200\nNov 2023: 161\nDec 2023: 198'),
  0.273825466632843),
 (Document(metadata={'row': 1, 'source': '/content/VolumeSheet.csv'}, page_content='Tenant: Apollo\nWorkflow: Claims\nStatus: Success\nJan 2023: 239\nFeb 2023: 239\nMar 2023: 899\nApr 2023: 192\nMay 2023: 164\nJun 2023: 103\nJul 2023: 152\nAug 2023: 149\nSep 2023: 192\nOct 2023: 164\nNov 2023: 103\nDec 2023: 152'),
  0.28

In [ ]:
#Define question
question = 'What are the total Success records for the Tenant Apollo for Claims workflow?'

#Fetch 3 documents from vector store related to question
vector_db.similarity_search_with_score(question, k=3)

[(Document(metadata={'row': 1, 'source': '/content/VolumeSheet.csv'}, page_content='Tenant: Apollo\nWorkflow: Claims\nStatus: Success\nJan 2023: 239\nFeb 2023: 239\nMar 2023: 899\nApr 2023: 192\nMay 2023: 164\nJun 2023: 103\nJul 2023: 152\nAug 2023: 149\nSep 2023: 192\nOct 2023: 164\nNov 2023: 103\nDec 2023: 152'),
  0.2380855679512024),
 (Document(metadata={'row': 1, 'source': '/content/VolumeSheet.csv'}, page_content='Tenant: Apollo\nWorkflow: Claims\nStatus: Success\nJan 2023: 239\nFeb 2023: 239\nMar 2023: 899\nApr 2023: 192\nMay 2023: 164\nJun 2023: 103\nJul 2023: 152\nAug 2023: 149\nSep 2023: 192\nOct 2023: 164\nNov 2023: 103\nDec 2023: 152'),
  0.2380855679512024),
 (Document(metadata={'row': 2, 'source': '/content/VolumeSheet.csv'}, page_content='Tenant: Apollo\nWorkflow: Claims\nStatus: Failed\nJan 2023: 100\nFeb 2023: 100\nMar 2023: 100\nApr 2023: 0\nMay 2023: 0\nJun 2023: 0\nJul 2023: 0\nAug 2023: 0\nSep 2023: 0\nOct 2023: 0\nNov 2023: 0\nDec 2023: 0'),
  0.24102582037448883)

#### <font color=FF595E>Biuld chain that will answer queestions over defined docs</font>

In [ ]:
from langchain.prompts import ChatPromptTemplate
# Prompt
template = """Answer the question based on the following context:
{context}

Question: {question}
"""

#Define rag_prompt from template
rag_prompt = ChatPromptTemplate.from_template(template)

#Print the promt to check it everything is ok
rag_prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Answer the question based on the following context:\n{context}\n\nQuestion: {question}\n'), additional_kwargs={})])

In [ ]:
#Define LLM

In [ ]:
RAG_llm = ChatOpenAI(model=GPT4)

In [ ]:
#Define Chain

In [ ]:
RAG_chain = rag_prompt | RAG_llm

In [ ]:
#Assign docs
docs = vector_db.similarity_search(question, k=3)

In [ ]:
#Chain to answer question based on defined docs
RAG_chain.invoke({"context":data,"question": question})

AIMessage(content='To find the total Success records for the Tenant Apollo for the Claims workflow, we need to sum the Success records reported for each month.\n\nFrom the provided documents, the Success records for Tenant Apollo in the Claims workflow are as follows:\n\n- Jan 2023: 239\n- Feb 2023: 239\n- Mar 2023: 899\n- Apr 2023: 192\n- May 2023: 164\n- Jun 2023: 103\n- Jul 2023: 152\n- Aug 2023: 149\n- Sep 2023: 192\n- Oct 2023: 164\n- Nov 2023: 103\n- Dec 2023: 152\n\nAdding these numbers gives us the total Success records for the Tenant Apollo for Claims workflow:\n\n239 + 239 + 899 + 192 + 164 + 103 + 152 + 149 + 192 + 164 + 103 + 152 = **2,738**\n\nTherefore, the total Success records for the Tenant Apollo for the Claims workflow are **2,738**.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 232, 'prompt_tokens': 5331, 'total_tokens': 5563, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning

#### <font color=FF595E>Composing the Retrieval-Augmented Generation Chain with dynamic retrieval</font>

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


# Create the Retrieval-Augmented Generation (RAG) chain with dynamic retrieval
rag_chain = (
    # Define the input variables for the chain
    {"context": retriever, "question": RunnablePassthrough()}
    # Pipe the input through the RAG prompt template
    | rag_prompt
    # Pass the formatted prompt to the language model (LLM)
    | RAG_llm
    # Parse the LLM's output using the StrOutputParser
    | StrOutputParser()
)

In [ ]:
#Invoke the chain

rag_chain.invoke("What is the volume of total transactions for Apollo for the year 2023?")


'To find the total volume of transactions for Apollo for the year 2023, we will sum the monthly transaction volumes listed in the first document, since it provides the "Total" status transactions for each month.\n\nHere are the monthly totals:\n\n- Jan 2023: 439\n- Feb 2023: 439\n- Mar 2023: 1099\n- Apr 2023: 194\n- May 2023: 200\n- Jun 2023: 161\n- Jul 2023: 198\n- Aug 2023: 153\n- Sep 2023: 194\n- Oct 2023: 200\n- Nov 2023: 161\n- Dec 2023: 198\n\nAdding these up:\n\n439 + 439 + 1099 + 194 + 200 + 161 + 198 + 153 + 194 + 200 + 161 + 198 = **3436**\n\nThe volume of total transactions for Apollo for the year 2023 is 3436.'

#### <font color=FF595E>Build RAG BOT chain prompt template</font>

In [ ]:
#Import required classes for prompt template

from langchain.prompts import PromptTemplate
from langchain.schema import HumanMessage

# Define the human prompt template
# Here we making sure that context will be passed to LLM before question will be added by human.
human_prompt = """Answer the question based on the following context: {context}

"""

In [ ]:
# Initialize the chat history
chat_history = ChatMessageHistory()

In [ ]:
# Define the question
question = "What is the volume of total transactions for Apollo for the year 2023?"

In [ ]:
# Define the question
question = "What is Fortis's total volume for April 2024 for Denials Processing workflow?"

In [ ]:
# Retrieve relevant context based on the question
context = vector_db.similarity_search(question, k=3)

In [ ]:
# Create a PromptTemplate from the human prompt
prompt_template = PromptTemplate.from_template(human_prompt)

In [ ]:
# Format the prompt with the retrieved context and question
formatted_prompt = prompt_template.format(context=context)

In [ ]:
# Create a HumanMessage with the formatted prompt
formatted_human_message = [HumanMessage(content=formatted_prompt)]

In [ ]:
# Define the RAG prompt template
rag_bot_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a data analyst. You have been provided a data regarding Healthcare facilities. You need to check the csv file related to volume and data. Total column refers to total number of records also known as total voloume of records/transactions or just total volume. The document contains differents status- Total (Total Records Processed), Completed (Success), Failed, Terminated records, In Progress. This is provided for the entitre year 2023 for each month separately. Tenant refers to the client and workflow refers to a specific workflow. One tenant can have multiple workflows as stated. You need to prepare bar graphs or line charts for the questions asked by the user"),
        *formatted_human_message,
        MessagesPlaceholder(variable_name="messages"),
    ]
)


In [ ]:
# Create the RAG LLM chain by piping the RAG prompt to the LLM
rag_bot_chain = rag_bot_prompt | RAG_llm

In [ ]:
#Define stop words for our chatbot
stop_words = ["exit", "quit", "stop"]

chat_history = ChatMessageHistory()


rag_chain_with_message_history = RunnableWithMessageHistory(
    rag_bot_chain,
    lambda session_id : chat_history,
    input_messages_key="messages",
    history_messages_key="chat_history"
)

# Perform chat turns
print("Starting the chat...")
while True:
    question = input("User: ")

    # Check if the user input matches a stop word
    if question.lower() in stop_words:
        print("Exiting the chat...")
        break

    # Retrieve relevant context based on the question
    context = vector_db.similarity_search(question, k=3)

    # Add a user message to the chat history
    chat_history.add_user_message(question)

    #Generate AI response
    ai_response = rag_chain_with_message_history.invoke({"messages": chat_history.messages}, {"configurable": {"session_id": chat_history}})

    # Add an AI response message to the chat history
    chat_history.add_ai_message(ai_response.content)

    #Display AI answer
    print(f"AI: {ai_response.content}")

Starting the chat...
User: For Tenant Apollo, give me all the total records for entire year 2023
AI: The documents you've provided don't include any data for Tenant Apollo. They only contain information related to Tenant Fortis, specifically for the Workflow: Claims, and their statuses across 2023. If you have any information regarding Apollo, please provide that for further assistance.
User: For Tenant Fortis, give me all the total records for entire year 2023
AI: For Tenant Fortis, focusing on the Workflow: Claims, here are the total records for each month in the entire year of 2023:

- January 2023: 400
- February 2023: 400
- March 2023: 1137
- April 2023: 1203
- May 2023: 1053
- June 2023: 1021
- July 2023: 929
- August 2023: 1084
- September 2023: 1121
- October 2023: 1124
- November 2023: 1304
- December 2023: 1255

These figures represent the total volume of records processed in each month for the year 2023.
User: exit
Exiting the chat...
